# Module

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import json
import os
import sys
import dotenv
import nest_asyncio
dotenv.load_dotenv()
import requests
import json
from glob import glob
import time
import pandas as pd
from tqdm import tqdm
from urllib.parse import unquote
from collections import defaultdict
from bs4 import BeautifulSoup

from pdf2image import convert_from_path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from PIL import Image, ImageDraw

# HTML 
* 하나의 PDF로 부터 나온 html tag는, id값이 1씩 increase 하도록 후처리
* 추후 해당 id를 활용해, json으로 부터 dounding_box와 page를 가져올 예정

In [45]:
# html 파일을 읽어서, id 값을 수정하는 함수
def change_html_tag_id(html_path, save_path):
    html = open(html_path).read()
    soup = BeautifulSoup(html, 'html.parser')
    re = 0
    for tag in soup.find_all():
        if tag.get('id'):
            tag['id'] = re
            re += 1
    with open(save_path, 'w') as f:    
        f.write(str(soup))  


In [51]:
html_paths = glob("../data/nursing/nursing_html/*")
for html_path in html_paths:
    file_name = os.path.basename(html_path)
    change_html_tag_id(html_path, f"../data/nursing/nursing_html2/{file_name}")

# JSON
* Document Parser로 부터 저장된 JSON 파일을 PDF 단위로 결합
* 결합시, id는 0부터 시작하여 HTML의 파일과 동일하게 매칭

In [2]:
def json_post_process(file_paths, save_folder_path):
    file_names = list(set([os.path.basename(file_path).split(".pdf")[0] for file_path in file_paths]))
    re = defaultdict(list)

    # 파일 이름이 같은 것들을 묶어준다.
    for file_name in file_names:
        for file_path in file_paths:
            if file_name in file_path:
                re[file_name].append(file_path)
    
    # 정렬 해 준다.
    for file_name in re:
        re[file_name] = sorted(re[file_name])
    
    for file_name in re:
        re_json = {"elements":[]}
        for file_path in re[file_name]:
            with open(file_path, "r") as f:
                data = json.load(f)
                re_json["elements"].extend(data["elements"])
        id_ = 0
        for r in re_json["elements"]:
            r["id"] = id_
            id_ += 1
            for b in r["bounding_box"]:
                b["x"] = b["x"]/1362
                b["y"] = b["y"]/1776
        
        with open(os.path.join(save_folder_path, file_name+".json"), 'w') as f:
            json.dump(re_json, f, indent=4)

In [3]:
json_post_process(glob("../data/nursing/nursing_downloads/*.json"), "../data/nursing/nursing_json")

# PDF to PNG
* PDF 파일을 PNG 이미지로 변환
* 형식 : 파일이름_page

In [13]:
pdf_paths = glob("../data/nursing/*.pdf")

In [14]:
pdf_paths

['../data/nursing/22장 신경계장애 대상자간호-(1).pdf',
 '../data/nursing/18장 요로계 장애 대상자 간호.pdf',
 '../data/nursing/14장 호흡기장애 대상자 간호-(2).pdf',
 '../data/nursing/17장 혈액계 장애 대상자 간호.pdf',
 '../data/nursing/21장 근골격계 장애.pdf',
 '../data/nursing/20장 유방장애 대상자 간호.pdf',
 '../data/nursing/27장 화상간호.pdf',
 '../data/nursing/25장 청각과 평형장애 대상자 간호.pdf',
 '../data/nursing/30 판권.pdf',
 '../data/nursing/26장 피부장애 대상자 간호.pdf',
 '../data/nursing/16장 혈관장애 대상자 간호.pdf',
 '../data/nursing/19장 남성 생식기계 대상자 간호.pdf',
 '../data/nursing/14장 호흡기장애 대상자 간호-(1).pdf',
 '../data/nursing/22장 신경계장애 대상자간호-(2).pdf',
 '../data/nursing/00 도입부(하권).pdf',
 '../data/nursing/23장 내분비장애 대상자 간호.pdf',
 '../data/nursing/24장 시각장애 대상자 간호.pdf',
 '../data/nursing/15장 심장장애 대상자 간호-(2).pdf',
 '../data/n

In [16]:
# for pdf_path in tqdm(pdf_paths):
#     images = convert_from_path(pdf_path, dpi=150)
#     file_name = os.path.basename(pdf_path).split(".pdf")[0]
    
#     if not os.path.exists(f"../data/nursing/nursing_images/{file_name}"):
#         os.makedirs(f"../data/nursing/nursing_images/{file_name}")

#     for i, image in enumerate(images):
#         image.save(f"../data/nursing/nursing_images/{file_name}/{i}.png")

100%|██████████| 19/19 [09:26<00:00, 29.82s/it]


# Bounding Box

In [23]:
title = "23장 내분비장애 대상자 간호"
t = """서맥, 고<br/>혈압, 고콜레스테롤혈증, 심장비대, 심부전(심장기능상<br/>실)이 나타나는지 관찰한다.</p><br/><p data-category="paragraph" id="1288" style="font-size:18px">또한 눈 주위나 안면의 부종, 멍한 표정, 두꺼운 혀,<br/>근육활동저하 등의 신체증상을 관찰한다. 저체온증, 호<br/>흡이나 심박동수 및 혈압감소 등의 활력징후를 자세히<br/>기록한다. 갑상샘호르몬제는 공복에 투여한다.</p><br/><p data-category="paragraph" id="1289" style="font-size:18px">당뇨병 대상자는 갑상샘저하증으로 인해 혈당치가 더<br/>불안정해지므로 주의 깊게 모니터한다. 또한 갑상샘장<br/>애의 중증 합병증을 확인한다(표 23-20).</p><p data-category="paragraph" id="1290" style="font-size:22px">(1) 합병증 예방</p><br/><p data-category="paragraph" id="1291" style="font-size:18px">갑상샘저하증은 신진대사율 감소와 무기력으로 움직<br/>이지 않아서 합병증이 초래되기 쉽다. 점액부종으로 심<br/>혈관계와 폐기능이 저하되어 운동과 활동에 제한을 받<br/>는다. 저산소증을 교정하기 위해 심호흡과 조기이상을<br/>권장한다. 또한 부동으로 인한 합병증을 예방하기 위해<br/>대상자가 활동에 참여하도록 격려한다. 로션과 크림을<br/>발라주어 피부건조를 완화하며 위생과 관리를 도와준<br/>다. 신체적·정신적 상태의 악화, 치료의 부작용, 점액부<br/>종의 합병증 등을 발견하기 위해 활력징후와 인지수준<br/>을 주의 깊게 관찰한다.</p><p data-category="paragraph" id="1292" style="font-size:20px">(2) 보온</p><br/><p data-category="paragraph" id="1293" style="font-size:18px">쉽게 오한과 추위를 느끼므로 편안하고 따뜻한 환경<br/>을 제공한다. 여분의 옷과 담요를 덮어주고 바람을 막<br/>아준다. 전기담요나 보온패드(heating pad)는 말초혈관<br/>이완, 체온상실, 혈관허탈을 일으킬 수 있기 때문에 피<br/>한다."""

In [24]:
# html 형식 str 로 부터 tag id 추출
def extract_tag_id(html_str):
    soup = BeautifulSoup(html_str, 'html.parser')
    re = []
    for tag in soup.find_all():
        if tag.get('id'):
            re.append(tag.get('id'))
    return re

In [25]:
tag_ids = extract_tag_id(t)
tag_ids

['1288', '1289', '1290', '1291', '1292', '1293']

In [26]:
with open(f"../data/nursing/nursing_json/{title}.json", "r") as f:
    json_data = json.load(f)

In [27]:
page_dict = defaultdict(list)
for tag_id in tag_ids:
    element = json_data["elements"][int(tag_id)]
    page_dict[element["page"]].append(element["bounding_box"])

In [28]:
tag_ids

['1288', '1289', '1290', '1291', '1292', '1293']

In [29]:
page_dict

defaultdict(list,
            {85: [[{'x': 0.513215859030837, 'y': 0.25844594594594594},
               {'x': 0.869309838472834, 'y': 0.25844594594594594},
               {'x': 0.869309838472834, 'y': 0.3333333333333333},
               {'x': 0.513215859030837, 'y': 0.3333333333333333}],
              [{'x': 0.5124816446402349, 'y': 0.33783783783783783},
               {'x': 0.8707782672540382, 'y': 0.33783783783783783},
               {'x': 0.8707782672540382, 'y': 0.3952702702702703},
               {'x': 0.5124816446402349, 'y': 0.3952702702702703}],
              [{'x': 0.5139500734214391, 'y': 0.4177927927927928},
               {'x': 0.6255506607929515, 'y': 0.4177927927927928},
               {'x': 0.6255506607929515, 'y': 0.4341216216216216},
               {'x': 0.5139500734214391, 'y': 0.4341216216216216}],
              [{'x': 0.5146842878120411, 'y': 0.4369369369369369},
               {'x': 0.8715124816446402, 'y': 0.4369369369369369},
               {'x': 0.87151248164464

In [30]:
# png 불러와서 바운딩 박스 그리고 시각화
def draw_bounding_box(file_name, page_dict):
    result = []
    for page_ in page_dict:
        image = Image.open(f"../data/nursing/nursing_images/{file_name}/{page_-1}.png")
        w, h = image.size
        draw = ImageDraw.Draw(image)
        for bbox in page_dict[page_]:
            x1 = bbox[0]["x"]*w
            y1 = bbox[0]["y"]*h
            x2 = bbox[1]["x"]*w
            y2 = bbox[2]["y"]*h
            draw.rectangle([x1, y1, x2, y2], outline=(255,0,0), width=2)
        image.save(f"./{file_name}_{page_-1}.png")
        print(f"SAVE - {file_name}_{page_-1}.png")

In [31]:
draw_bounding_box(title, page_dict)

SAVE - 23장 내분비장애 대상자 간호_84.png
